In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split as tts

In [2]:
from sklearn.linear_model import LinearRegression as LinReg
from sklearn.linear_model import Lasso        # regularizacion L1
from sklearn.linear_model import Ridge        # regularizacion L2
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor as RFR  
from sklearn.tree import ExtraTreeRegressor as ETR
from sklearn.ensemble import GradientBoostingRegressor as GBR
from xgboost import XGBRegressor as XGBR
from catboost import CatBoostRegressor as CTR
from lightgbm import LGBMRegressor as LGBMR

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
pd.set_option('display.max_rows', None)

In [5]:
boe=pd.read_csv('../data/BOE_def.csv')

In [6]:
boe.fecha_de_conclusion.unique()

array(['08-12-2022', '07-12-2022', '06-12-2022', ..., '06-02-2016',
       '27-01-2016', '10-01-2016'], dtype=object)

In [7]:
import regex as re

In [8]:
def limpiar_(fecha_de_inicio):
    fecha_de_inicio=re.findall('\d{4}', fecha_de_inicio)
    fecha_de_inicio=' '.join(fecha_de_inicio)
    return fecha_de_inicio

In [9]:
boe.fecha_de_inicio=boe.fecha_de_inicio.apply(limpiar_)

In [10]:
boe.fecha_de_conclusion=boe.fecha_de_conclusion.apply(limpiar_)

In [11]:
boe.head()

,identificador,tipo_de_subasta,cuenta_expediente,fecha_de_inicio,fecha_de_conclusion,cantidad_reclamada,anuncio_BOE,valor_subasta,tasacion,puja_minima,tramos_entre_pujas,importe_del_deposito,direccion,codigo_postal,localidad,provincia,puja,pm2
0,SUB-JA-2022-204013,JUDICIAL EN VÍA DE APREMIO,4743 0000 05 1018 20,2022,2022,7145,BOE-B-2022-35270,108482,0,0,2169,5424,Urbanización Els Racons nº 5. partida Michafiga,46220,Picassent,Valencia/València,58580,1088
1,SUB-JA-2022-196067,JUDICIAL EN VÍA DE APREMIO,2368 0000 06 0191 20,2022,2022,142298,BOE-B-2022-35269,305832,0,0,6116,15291,"CALLE GABRIEL GARCÍA MÁRQUEZ, 31",28980,PARLA,Madrid,177382,1721
2,SUB-JA-2022-204060,JUDICIAL EN VÍA DE APREMIO,2995 0000 06 0456 16,2022,2022,55933,BOE-B-2022-35263,247902,0,0,0,12395,VIVIENDA 1 BLOQUE 19 CONJUNTO RESIDENCIAL HACI...,29600,Marbella,Málaga,252000,1721
3,SUB-JA-2022-198972,JUDICIAL EN VÍA DE APREMIO,3997 0000 06 1661 20,2022,2022,210425,BOE-B-2022-28879,826272,0,0,16525,41313,"AVENIDA DE JACARANDA, 5, URBANIZACION CLUB DE ...",41930,BORMUJOS,Sevilla,512288,1557
4,SUB-JA-2022-203383,JUDICIAL EN VÍA DE APREMIO,5418 0000 06 0169 21,2022,2022,34853,BOE-B-2022-35253,121684,121684,45978,2433,6084,"Calle Palma de Mallorca, nº 13, bloque 19, 3º ...",35016,Las Palmas de Gran Canaria,Las Palmas,45978,1130


In [12]:
boe.drop(columns=['identificador','tipo_de_subasta','cuenta_expediente',
                  'anuncio_BOE','tasacion','puja_minima','tramos_entre_pujas',
                 'importe_del_deposito','direccion','localidad'], inplace=True)

In [13]:
boe.head()

,fecha_de_inicio,fecha_de_conclusion,cantidad_reclamada,valor_subasta,codigo_postal,provincia,puja,pm2
0,2022,2022,7145,108482,46220,Valencia/València,58580,1088
1,2022,2022,142298,305832,28980,Madrid,177382,1721
2,2022,2022,55933,247902,29600,Málaga,252000,1721
3,2022,2022,210425,826272,41930,Sevilla,512288,1557
4,2022,2022,34853,121684,35016,Las Palmas,45978,1130


In [14]:
boe.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6154 entries, 0 to 6153
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   fecha_de_inicio      6154 non-null   object
 1   fecha_de_conclusion  6154 non-null   object
 2   cantidad_reclamada   6154 non-null   int64 
 3   valor_subasta        6154 non-null   int64 
 4   codigo_postal        6154 non-null   int64 
 5   provincia            6154 non-null   object
 6   puja                 6154 non-null   int64 
 7   pm2                  6154 non-null   int64 
dtypes: int64(5), object(3)
memory usage: 1.4 MB


In [15]:
boe['fecha_de_inicio'] = boe['fecha_de_inicio'].astype(int)

In [16]:
boe['fecha_de_conclusion'] = boe['fecha_de_conclusion'].astype(int)

In [17]:
#eliminamos esta columna porque el precio se cierra en la fecha de conclusion y varia segun el año final.
boe.drop(columns=['fecha_de_inicio'], inplace=True)

In [18]:
#eliminamos codigo postal? con eso hemos sacado el pm2, no nos interesa más. Ademas contamos con la provincia.
boe.drop(columns=['codigo_postal'], inplace=True)

In [19]:
boe.head()

,fecha_de_conclusion,cantidad_reclamada,valor_subasta,provincia,puja,pm2
0,2022,7145,108482,Valencia/València,58580,1088
1,2022,142298,305832,Madrid,177382,1721
2,2022,55933,247902,Málaga,252000,1721
3,2022,210425,826272,Sevilla,512288,1557
4,2022,34853,121684,Las Palmas,45978,1130


In [20]:
#creamos new colum con la diferencia entre la deuda(cantidad_reclamada) y el valor que le da el juez(valor_subasta)
boe['diff'] = boe['valor_subasta'].sub(boe['cantidad_reclamada'], axis = 0)

In [21]:
boe.head()

,fecha_de_conclusion,cantidad_reclamada,valor_subasta,provincia,puja,pm2,diff
0,2022,7145,108482,Valencia/València,58580,1088,101337
1,2022,142298,305832,Madrid,177382,1721,163534
2,2022,55933,247902,Málaga,252000,1721,191969
3,2022,210425,826272,Sevilla,512288,1557,615847
4,2022,34853,121684,Las Palmas,45978,1130,86831


In [22]:
boe.drop(columns=['cantidad_reclamada','valor_subasta'], inplace=True)

In [23]:
boe.head()

,fecha_de_conclusion,provincia,puja,pm2,diff
0,2022,Valencia/València,58580,1088,101337
1,2022,Madrid,177382,1721,163534
2,2022,Málaga,252000,1721,191969
3,2022,Sevilla,512288,1557,615847
4,2022,Las Palmas,45978,1130,86831


In [24]:
boe=pd.get_dummies(boe, columns=['fecha_de_conclusion','provincia'], drop_first=True)

boe.head()

,puja,pm2,diff,fecha_de_conclusion_2017,fecha_de_conclusion_2018,fecha_de_conclusion_2019,fecha_de_conclusion_2020,fecha_de_conclusion_2021,fecha_de_conclusion_2022,provincia_Albacete,provincia_Alicante,provincia_Alicante/Alacant,provincia_Almería,provincia_Araba/Álava,provincia_Asturias,provincia_Badajoz,provincia_Baleares,provincia_Barcelona,provincia_Bizkaia,provincia_Burgos,provincia_Cantabria,provincia_Castellón,provincia_Castellón/Castelló,provincia_Ceuta,provincia_Ciudad Real,provincia_Cuenca,provincia_Cáceres,provincia_Cádiz,provincia_Córdoba,provincia_Gerona,provincia_Gipuzkoa,provincia_Girona,provincia_Granada,provincia_Guadalajara,provincia_Guipúzcoa,provincia_Huelva,provincia_Huesca,provincia_Illes Balears,provincia_Jaén,provincia_La Coruña,provincia_La Rioja,provincia_Las Palmas,provincia_León,provincia_Lleida,provincia_Lugo,provincia_Lérida,provincia_Madrid,provincia_Melilla,provincia_Murcia,provincia_Málaga,provincia_Navarra,provincia_Orense,provincia_Ourense,provincia_Palencia,provincia_Pontevedra,provincia_Salamanca,provincia_Santa Cruz de Tenerife,provincia_Segovia,provincia_Sevilla,provincia_Soria,provincia_Tarragona,provincia_Teruel,provincia_Toledo,provincia_Valencia,provincia_Valencia/València,provincia_Valladolid,provincia_Vizcaya,provincia_Zamora,provincia_Zaragoza,provincia_Álava,provincia_Ávila
0,58580,1088,101337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,177382,1721,163534,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,252000,1721,191969,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,512288,1557,615847,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,45978,1130,86831,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [25]:
boe.head()

,puja,pm2,diff,fecha_de_conclusion_2017,fecha_de_conclusion_2018,fecha_de_conclusion_2019,fecha_de_conclusion_2020,fecha_de_conclusion_2021,fecha_de_conclusion_2022,provincia_Albacete,provincia_Alicante,provincia_Alicante/Alacant,provincia_Almería,provincia_Araba/Álava,provincia_Asturias,provincia_Badajoz,provincia_Baleares,provincia_Barcelona,provincia_Bizkaia,provincia_Burgos,provincia_Cantabria,provincia_Castellón,provincia_Castellón/Castelló,provincia_Ceuta,provincia_Ciudad Real,provincia_Cuenca,provincia_Cáceres,provincia_Cádiz,provincia_Córdoba,provincia_Gerona,provincia_Gipuzkoa,provincia_Girona,provincia_Granada,provincia_Guadalajara,provincia_Guipúzcoa,provincia_Huelva,provincia_Huesca,provincia_Illes Balears,provincia_Jaén,provincia_La Coruña,provincia_La Rioja,provincia_Las Palmas,provincia_León,provincia_Lleida,provincia_Lugo,provincia_Lérida,provincia_Madrid,provincia_Melilla,provincia_Murcia,provincia_Málaga,provincia_Navarra,provincia_Orense,provincia_Ourense,provincia_Palencia,provincia_Pontevedra,provincia_Salamanca,provincia_Santa Cruz de Tenerife,provincia_Segovia,provincia_Sevilla,provincia_Soria,provincia_Tarragona,provincia_Teruel,provincia_Toledo,provincia_Valencia,provincia_Valencia/València,provincia_Valladolid,provincia_Vizcaya,provincia_Zamora,provincia_Zaragoza,provincia_Álava,provincia_Ávila
0,58580,1088,101337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,177382,1721,163534,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,252000,1721,191969,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,512288,1557,615847,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,45978,1130,86831,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [26]:
boe.pm2=LabelEncoder().fit_transform(boe.pm2)

In [27]:
boe.head()

,puja,pm2,diff,fecha_de_conclusion_2017,fecha_de_conclusion_2018,fecha_de_conclusion_2019,fecha_de_conclusion_2020,fecha_de_conclusion_2021,fecha_de_conclusion_2022,provincia_Albacete,provincia_Alicante,provincia_Alicante/Alacant,provincia_Almería,provincia_Araba/Álava,provincia_Asturias,provincia_Badajoz,provincia_Baleares,provincia_Barcelona,provincia_Bizkaia,provincia_Burgos,provincia_Cantabria,provincia_Castellón,provincia_Castellón/Castelló,provincia_Ceuta,provincia_Ciudad Real,provincia_Cuenca,provincia_Cáceres,provincia_Cádiz,provincia_Córdoba,provincia_Gerona,provincia_Gipuzkoa,provincia_Girona,provincia_Granada,provincia_Guadalajara,provincia_Guipúzcoa,provincia_Huelva,provincia_Huesca,provincia_Illes Balears,provincia_Jaén,provincia_La Coruña,provincia_La Rioja,provincia_Las Palmas,provincia_León,provincia_Lleida,provincia_Lugo,provincia_Lérida,provincia_Madrid,provincia_Melilla,provincia_Murcia,provincia_Málaga,provincia_Navarra,provincia_Orense,provincia_Ourense,provincia_Palencia,provincia_Pontevedra,provincia_Salamanca,provincia_Santa Cruz de Tenerife,provincia_Segovia,provincia_Sevilla,provincia_Soria,provincia_Tarragona,provincia_Teruel,provincia_Toledo,provincia_Valencia,provincia_Valencia/València,provincia_Valladolid,provincia_Vizcaya,provincia_Zamora,provincia_Zaragoza,provincia_Álava,provincia_Ávila
0,58580,436,101337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,177382,805,163534,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,252000,805,191969,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,512288,715,615847,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,45978,467,86831,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [28]:
boe.diference.unique()

AttributeError: 'DataFrame' object has no attribute 'diference'

In [29]:
boe=boe.rename(columns = {'diff':'diference'})

In [30]:
boe.diference=LabelEncoder().fit_transform(boe.diference)

In [31]:
boe.head()

,puja,pm2,diference,fecha_de_conclusion_2017,fecha_de_conclusion_2018,fecha_de_conclusion_2019,fecha_de_conclusion_2020,fecha_de_conclusion_2021,fecha_de_conclusion_2022,provincia_Albacete,provincia_Alicante,provincia_Alicante/Alacant,provincia_Almería,provincia_Araba/Álava,provincia_Asturias,provincia_Badajoz,provincia_Baleares,provincia_Barcelona,provincia_Bizkaia,provincia_Burgos,provincia_Cantabria,provincia_Castellón,provincia_Castellón/Castelló,provincia_Ceuta,provincia_Ciudad Real,provincia_Cuenca,provincia_Cáceres,provincia_Cádiz,provincia_Córdoba,provincia_Gerona,provincia_Gipuzkoa,provincia_Girona,provincia_Granada,provincia_Guadalajara,provincia_Guipúzcoa,provincia_Huelva,provincia_Huesca,provincia_Illes Balears,provincia_Jaén,provincia_La Coruña,provincia_La Rioja,provincia_Las Palmas,provincia_León,provincia_Lleida,provincia_Lugo,provincia_Lérida,provincia_Madrid,provincia_Melilla,provincia_Murcia,provincia_Málaga,provincia_Navarra,provincia_Orense,provincia_Ourense,provincia_Palencia,provincia_Pontevedra,provincia_Salamanca,provincia_Santa Cruz de Tenerife,provincia_Segovia,provincia_Sevilla,provincia_Soria,provincia_Tarragona,provincia_Teruel,provincia_Toledo,provincia_Valencia,provincia_Valencia/València,provincia_Valladolid,provincia_Vizcaya,provincia_Zamora,provincia_Zaragoza,provincia_Álava,provincia_Ávila
0,58580,436,3988,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,177382,805,4951,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,252000,805,5186,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,512288,715,5790,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,45978,467,3602,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [32]:
boe_x=boe.drop('puja', axis=1)
boe_y=boe.puja

In [37]:
X_train, X_test, y_train, y_test = tts(boe_x, boe_y,train_size=0.8, test_size=0.2,random_state =42)

**Iniciamos los modelos**

In [38]:
linreg=LinReg()
lasso=Lasso()
ridge=Ridge()
elastic=ElasticNet()
svr=SVR()
rfr=RFR()
etr=ETR()
gbr=GBR()
xgbr=XGBR()
ctr=CTR()
lgbmr=LGBMR()

**Entrenamos modelos**

In [39]:
linreg.fit(X_train, y_train)
lasso.fit(X_train, y_train)
ridge.fit(X_train, y_train)
elastic.fit(X_train, y_train)
svr.fit(X_train, y_train)
rfr.fit(X_train, y_train)
etr.fit(X_train, y_train)
gbr.fit(X_train, y_train)
xgbr.fit(X_train, y_train)
ctr.fit(X_train, y_train)
lgbmr.fit(X_train, y_train)

Learning rate set to 0.052669
0:	learn: 617469.6708823	total: 1.03ms	remaining: 1.03s
1:	learn: 610462.3060275	total: 2.23ms	remaining: 1.11s
2:	learn: 604867.9951856	total: 3.13ms	remaining: 1.04s
3:	learn: 598063.1762080	total: 4.07ms	remaining: 1.01s
4:	learn: 591370.9067686	total: 5.07ms	remaining: 1.01s
5:	learn: 584842.5435599	total: 6.17ms	remaining: 1.02s
6:	learn: 578417.2266750	total: 7.13ms	remaining: 1.01s
7:	learn: 572117.9335636	total: 8.02ms	remaining: 994ms
8:	learn: 565892.0387577	total: 9.09ms	remaining: 1s
9:	learn: 559801.6226600	total: 10.2ms	remaining: 1.01s
10:	learn: 553797.1073860	total: 11.3ms	remaining: 1.01s
11:	learn: 547894.0969872	total: 12.4ms	remaining: 1.02s
12:	learn: 542102.2873368	total: 13.5ms	remaining: 1.02s
13:	learn: 536445.9638555	total: 14.6ms	remaining: 1.03s
14:	learn: 530856.8730221	total: 15.6ms	remaining: 1.02s
15:	learn: 525364.8828629	total: 16.7ms	remaining: 1.03s
16:	learn: 519826.8656363	total: 17.9ms	remaining: 1.04s
17:	learn: 514

175:	learn: 199890.6991940	total: 192ms	remaining: 897ms
176:	learn: 199125.5469498	total: 193ms	remaining: 896ms
177:	learn: 198464.0965253	total: 194ms	remaining: 894ms
178:	learn: 197715.1306138	total: 195ms	remaining: 892ms
179:	learn: 197058.1103466	total: 196ms	remaining: 891ms
180:	learn: 196414.8955843	total: 197ms	remaining: 890ms
181:	learn: 195679.6301408	total: 198ms	remaining: 888ms
182:	learn: 194958.9653739	total: 199ms	remaining: 887ms
183:	learn: 194090.7539210	total: 200ms	remaining: 885ms
184:	learn: 193416.1165847	total: 201ms	remaining: 884ms
185:	learn: 192460.9376491	total: 201ms	remaining: 882ms
186:	learn: 191843.4325722	total: 203ms	remaining: 881ms
187:	learn: 191166.4114208	total: 210ms	remaining: 906ms
188:	learn: 190489.8479247	total: 211ms	remaining: 906ms
189:	learn: 189881.0672068	total: 212ms	remaining: 905ms
190:	learn: 189282.8751992	total: 213ms	remaining: 904ms
191:	learn: 188698.9445530	total: 215ms	remaining: 904ms
192:	learn: 188099.4444999	tota

359:	learn: 133241.3942700	total: 385ms	remaining: 684ms
360:	learn: 133030.6938768	total: 386ms	remaining: 683ms
361:	learn: 132901.1006766	total: 387ms	remaining: 682ms
362:	learn: 132772.2416014	total: 388ms	remaining: 680ms
363:	learn: 132646.5175595	total: 389ms	remaining: 679ms
364:	learn: 132524.0497238	total: 389ms	remaining: 677ms
365:	learn: 132287.1000164	total: 390ms	remaining: 676ms
366:	learn: 132139.0500055	total: 391ms	remaining: 675ms
367:	learn: 132019.4140484	total: 392ms	remaining: 674ms
368:	learn: 131894.1939765	total: 393ms	remaining: 673ms
369:	learn: 131777.2331810	total: 394ms	remaining: 672ms
370:	learn: 131573.6126025	total: 396ms	remaining: 671ms
371:	learn: 131453.2299985	total: 397ms	remaining: 669ms
372:	learn: 131254.8841669	total: 398ms	remaining: 668ms
373:	learn: 131113.6999620	total: 399ms	remaining: 667ms
374:	learn: 131002.1939836	total: 400ms	remaining: 666ms
375:	learn: 130893.6152006	total: 401ms	remaining: 665ms
376:	learn: 130661.3889624	tota

548:	learn: 112074.2681978	total: 577ms	remaining: 474ms
549:	learn: 112000.7981800	total: 578ms	remaining: 473ms
550:	learn: 111926.2934742	total: 579ms	remaining: 472ms
551:	learn: 111832.7702286	total: 580ms	remaining: 471ms
552:	learn: 111692.1037272	total: 581ms	remaining: 470ms
553:	learn: 111665.8482861	total: 582ms	remaining: 469ms
554:	learn: 111638.5476154	total: 583ms	remaining: 468ms
555:	learn: 111563.2307940	total: 584ms	remaining: 466ms
556:	learn: 111425.2113411	total: 585ms	remaining: 465ms
557:	learn: 111249.0345248	total: 586ms	remaining: 464ms
558:	learn: 111190.3833272	total: 587ms	remaining: 463ms
559:	learn: 111127.3180401	total: 588ms	remaining: 462ms
560:	learn: 111067.0571353	total: 589ms	remaining: 461ms
561:	learn: 111040.9904403	total: 590ms	remaining: 460ms
562:	learn: 111015.6579647	total: 591ms	remaining: 459ms
563:	learn: 110977.5251459	total: 592ms	remaining: 458ms
564:	learn: 110947.0174072	total: 593ms	remaining: 457ms
565:	learn: 110778.2156418	tota

736:	learn: 101180.1708821	total: 770ms	remaining: 275ms
737:	learn: 101106.9255436	total: 771ms	remaining: 274ms
738:	learn: 101060.0818799	total: 772ms	remaining: 273ms
739:	learn: 101018.0525960	total: 773ms	remaining: 272ms
740:	learn: 101003.3371790	total: 774ms	remaining: 271ms
741:	learn: 100991.0187708	total: 775ms	remaining: 270ms
742:	learn: 100980.4742779	total: 776ms	remaining: 269ms
743:	learn: 100961.0513735	total: 777ms	remaining: 268ms
744:	learn: 100819.9304437	total: 778ms	remaining: 266ms
745:	learn: 100759.1174310	total: 779ms	remaining: 265ms
746:	learn: 100689.0414935	total: 780ms	remaining: 264ms
747:	learn: 100655.5987429	total: 781ms	remaining: 263ms
748:	learn: 100641.1146219	total: 782ms	remaining: 262ms
749:	learn: 100518.6654074	total: 783ms	remaining: 261ms
750:	learn: 100497.9831857	total: 784ms	remaining: 260ms
751:	learn: 100476.2632689	total: 785ms	remaining: 259ms
752:	learn: 100445.2224714	total: 786ms	remaining: 258ms
753:	learn: 100401.8775734	tota

924:	learn: 92638.6699913	total: 963ms	remaining: 78.1ms
925:	learn: 92586.6540682	total: 964ms	remaining: 77ms
926:	learn: 92560.9128288	total: 965ms	remaining: 76ms
927:	learn: 92497.6982553	total: 966ms	remaining: 74.9ms
928:	learn: 92482.0482188	total: 967ms	remaining: 73.9ms
929:	learn: 92383.1704811	total: 968ms	remaining: 72.9ms
930:	learn: 92349.2186505	total: 969ms	remaining: 71.8ms
931:	learn: 92302.2612508	total: 970ms	remaining: 70.8ms
932:	learn: 92279.3000019	total: 971ms	remaining: 69.8ms
933:	learn: 92263.3371535	total: 973ms	remaining: 68.7ms
934:	learn: 92188.3983367	total: 974ms	remaining: 67.7ms
935:	learn: 92125.1054791	total: 975ms	remaining: 66.7ms
936:	learn: 92053.5295394	total: 976ms	remaining: 65.6ms
937:	learn: 92042.7116065	total: 977ms	remaining: 64.6ms
938:	learn: 91981.3377055	total: 978ms	remaining: 63.5ms
939:	learn: 91883.4039807	total: 979ms	remaining: 62.5ms
940:	learn: 91839.0181216	total: 980ms	remaining: 61.4ms
941:	learn: 91782.7502271	total: 98

LGBMRegressor()

In [40]:
from sklearn.metrics import mean_squared_error as mse

In [41]:
from sklearn.metrics import r2_score as r2

In [42]:
#Import Lazypredict and all libraries
import lazypredict
from lazypredict.Supervised import LazyRegressor
from sklearn.model_selection import train_test_split
import os
import pandas as pd
import numpy as np

In [43]:
lazy = LazyRegressor(verbose=0,ignore_warnings=True, custom_metric=None)
models, predictions = lazy.fit(X_train, X_test, y_train, y_test)

100%|███████████████████████████████████████████| 42/42 [21:11<00:00, 30.26s/it]


In [44]:
print(models)

                               Adjusted R-Squared  R-Squared        RMSE  \
Model                                                                      
XGBRegressor                                 0.31       0.35   296429.27   
GradientBoostingRegressor                    0.30       0.34   298755.83   
HistGradientBoostingRegressor                0.29       0.33   301822.19   
RandomForestRegressor                        0.24       0.29   311499.81   
BaggingRegressor                             0.23       0.27   314367.17   
LGBMRegressor                                0.21       0.25   319227.46   
ExtraTreesRegressor                          0.20       0.25   320328.27   
DecisionTreeRegressor                        0.20       0.24   320640.99   
KNeighborsRegressor                          0.12       0.17   335563.03   
AdaBoostRegressor                            0.06       0.11   347404.84   
ExtraTreeRegressor                           0.05       0.10   349821.16   
PoissonRegre